## Old way - dictionary 

In [ ]:
## Create new dataset
dataset_197k = []
NEW_SEQUENCE_LENGTH = 196_608
max_steps = 2

for i, batch in tqdm(enumerate(human_dataset)):
    batch_197k = {}
    # 1 from the sequence 131k get the sequence 197k
    interval_test = get_interval_from_sequence(batch["sequence"])
    sequence_197k = one_hot_encode(fasta_extractor.extract(interval_test.resize(NEW_SEQUENCE_LENGTH)))
    batch_197k["sequence"] = tf.constant(sequence_197k[np.newaxis])
    
    # add same real targets
    batch_197k["target"] = batch["target"]
    dataset_197k.append(batch_197k)
    if max_steps is not None and i > max_steps:
        break

file = os.path.join(outputdir,'new_dataset_197k_valid.h5')

# Step 2
with open(file, 'wb') as config_dictionary_file:
    pickle.dump(dataset_197k, config_dictionary_file)

In [ ]:
dataset_197k[1]["sequence"]

In [ ]:
SEQUENCE_LENGHT = 393_216
## pad the sequence with Ns (anyways ignored by the model)
def pad_one_hot(sequence_one_hot, NEW_SIZE):
    ADD_ENDS = int((NEW_SIZE - sequence_one_hot.shape[0])/2)
    pad_zero = np.tile(np.array([0., 0., 0., 0.]), (ADD_ENDS, 1))
    padded_left = np.append(pad_zero,sequence_one_hot, axis=0)
    pad_sequence = np.append(padded_left,pad_zero, axis=0)
    return(pad_sequence)

In [ ]:
# try predictions
def evaluate_model_all_sequences_mod(model, dataset, head, max_steps=None):
    
    metric = MetricDict({'PearsonR': PearsonR(reduce_axis=(0,1))})
    
    def predict(x):
        padded_sequence = pad_one_hot(np.squeeze(x.numpy(), axis=0), SEQUENCE_LENGHT)[np.newaxis]
        predictions = model.predict_on_batch(padded_sequence)[head]
        return tf.convert_to_tensor(predictions, dtype=tf.float32)
    
    i = 0 
    for i, batch in enumerate(dataset_197k): 
        if max_steps is not None and i > max_steps:
            break
        i = i+1
        prediction = predict(batch['sequence'])
        metric.update_state(batch['target'], prediction)

    return metric.result()


In [ ]:
metrics_human = evaluate_model_all_sequences_mod(model,
                               dataset=dataset_197k[0:1],
                               head='human',
                               max_steps=2)

In [ ]:
min(metrics_human["PearsonR"].numpy())

In [ ]:
metrics_human

## Refined creation with tfr records

In [ ]:
# Update metadata
metadata_human_197k = get_metadata("human")
metadata_human_197k['seq_length'] = NEW_SEQUENCE_LENGTH

In [ ]:
def predict(x, head):
    padded_sequence = pad_one_hot(x, SEQUENCE_LENGHT)[np.newaxis]
    predictions = model.predict_on_batch(padded_sequence)[head]
    return tf.convert_to_tensor(predictions, dtype=tf.float32)

In [22]:
# Test entry 
for i, batch in tqdm(enumerate(human_dataset)):
    print(i)
    first_dataset_entry = batch
    break
#interval_test = get_interval_from_sequence(first_dataset_entry)
first_dataset_entry
#target_one = predict(first_dataset_entry["sequence"], "human")

0it [00:05, ?it/s]

0


{'sequence': <tf.Tensor: shape=(1, 131072, 4), dtype=float32, numpy=
 array([[[1., 0., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 1., 0.],
         ...,
         [0., 0., 0., 1.],
         [0., 0., 1., 0.],
         [1., 0., 0., 0.]]], dtype=float32)>,
 'target': <tf.Tensor: shape=(1, 896, 5313), dtype=float32, numpy=
 array([[[0.09924316, 0.0927124 , 0.01834106, ..., 0.        ,
          0.        , 0.        ],
         [0.11126709, 0.1685791 , 0.03396606, ..., 0.        ,
          0.984375  , 0.        ],
         [0.14318848, 0.23217773, 0.01850891, ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.00662994, 0.01672363, 0.00756454, ..., 0.01852417,
          0.11566162, 0.        ],
         [0.00411224, 0.00155735, 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.06958008, 0.03845215, 0.04312134, ..., 0.        ,
          0.        , 0.        ]]], dtype=float32)>}

### Try to add targets into tfr record (serialized!) 

In [23]:
sequence_one = first_dataset_entry["sequence"]
sequence_one

<tf.Tensor: shape=(1, 131072, 4), dtype=float32, numpy=
array([[[1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        ...,
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.]]], dtype=float32)>

In [ ]:
target_one = predict(np.squeeze(first_dataset_entry["sequence"]), "human")
target_one

In [185]:
def _array_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.numpy().tobytes()]))

def serialize_example(sequence, target):

    feature = {
      'sequence':  _array_feature(sequence),
      'target':  _array_feature(target),
    }

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()


def tf_serialize_example(sequence, target):
    tf_string = tf.py_function(serialize_example,(sequence, target), tf.string) 
    return tf.reshape(tf_string, ()) 

In [193]:
def generator():
    for features in features_dataset:
        yield serialize_example(*features)